In [62]:
import plotly.express as px
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"
#!python3 -mpip install prophet >> /dev/null
import requests
import pandas as pd
import numpy as np
from prophet import Prophet
import logging
import plotly.graph_objs as go
#logging.basicConfig(level=logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
r = requests.get("http://127.0.0.1:8000/summary")
r = r.json()
df = pd.DataFrame(r)
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

<style>
.container { width:100% !important; }
</style>

In [76]:
import plotly.graph_objs as go

m = Prophet(seasonality_mode="multiplicative", weekly_seasonality=False, daily_seasonality=False).fit(df)
future = m.make_future_dataframe(periods=0, freq='M')
forecast = m.predict(future)

# Calculate total report count
total_report_count = forecast['yhat'].sum()

# Get the 'yhat' value for the last month
last_month_yhat = forecast['yhat'].iloc[-1]

# Calculate the change in monthly reporting
change_in_monthly_reporting = forecast['yhat'].iloc[-1] - forecast['yhat'].iloc[-2]

# Determine the color based on the change
color = "green" if change_in_monthly_reporting >= 0 else "red"

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode="number",
    title={'text': "Total Report Count"},
    value=total_report_count,
    gauge={
        'axis': {'visible': False}},
    domain={'x': [0.0, 0.5], 'y': [0, 1]}))

fig.add_trace(go.Indicator(
    mode="delta",
    title= { "text" : "Change in Monthly Reporting" },
    value=change_in_monthly_reporting,
    delta = {'reference': 0, 'increasing': {'color': color}, 'decreasing': {'color': color}},
    domain={'x': [0.5, 1.0], 'y': [0, 1]}))

fig.update_layout(
    grid={'rows': 1, 'columns': 2, 'pattern': "independent"},
    template={'data': {'indicator': [{
        'mode': "number+delta+gauge",
        'delta': {'reference': 90}}]
    }},
    height=200)  # Set the height of the figure to the minimum

In [64]:

# Identify points outside the expected range
forecast['outlier'] = (df['y'] < forecast['yhat_lower']) | (df['y'] > forecast['yhat_upper'])

# Create a scatter plot for the forecast
fig = go.Figure()

# Set the background color to white
fig.update_layout(plot_bgcolor='white',
                  # paper_bgcolor='white'
                   )


fig.add_trace(go.Scatter(x=forecast['ds'], 
                         y=forecast['yhat'], 
                         mode='lines', 
                         hovertemplate='<b>Date</b>: %{x}<br>'+'<b>Report Count</b>: %{y}<br>',
                         name='Estimated'))
fig.add_trace(go.Scatter(x=forecast['ds'],
                        y=forecast['yhat_upper'],
                         mode='lines',
                         name='yhat_upper',
                         showlegend=False, 
                         hoverinfo='none', 
                         line=dict(width=0)))
fig.add_trace(go.Scatter(x=forecast['ds'],
                        y=forecast['yhat_lower'],
                        mode='lines', 
                        name='yhat_lower', 
                        showlegend=False, 
                        hoverinfo='none', 
                        line=dict(width=0)))
fig.add_trace(go.Scatter(x=forecast['ds'], 
                         y=df['y'], 
                         mode='markers',  
                         hovertemplate='<b>Date</b>: %{x}<br>'+'<b>Report Count</b>: %{y}<br>',
                        name='Report Count', 
                        marker=dict(color='black',
                                     size=4)))

# Add markers for the outliers
outliers = forecast[forecast['outlier']]
fig.add_trace(go.Scatter(x=outliers['ds'], y=df['y'][outliers.index], mode='markers', marker=dict(color='rgb(255, 69, 0)', size=5),  hovertemplate='<b>Date</b>: %{x}<br>'+'<b>Report Count</b>: %{y}<br>', name='Report Count'))

# Fill the area between yhat_upper and yhat_lower with the same color
fig.update_traces(fill='tonexty', fillcolor='rgba(173,216,230,0.5)', selector=dict(name='yhat_lower'))

# Set the range of the y-axis to start at 0 and end at 10% more than the top 'y' value
max_y = max(df['y'])
fig.update_yaxes(range=[0, max_y + 0.15 * max_y])

# Remove the background shading but leave the grid lines
#fig.update_layout( paper_bgcolor='white')

# Change the grid lines to be light grey
fig.update_xaxes(gridcolor='lightgrey')
fig.update_yaxes(gridcolor='lightgrey')

# Add "Report Count" to the y-axis
fig.update_yaxes(title_text="Report Count")

# Add an overall trend line as a dotted line
#fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['trend'], mode='lines', line=dict(dash='dot'), name='Overall Trend'))

# Convert the datetime objects to the number of seconds since the Unix epoch
df['ds_seconds'] = (pd.to_datetime(df['ds']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# Fit a line to the data
slope, intercept = np.polyfit(df['ds_seconds'], df['y'], 1)
line = slope * df['ds_seconds'] + intercept
# Add the line to the plot
fig.add_trace(go.Scatter(x=df['ds'], 
                         y=line, 
                         mode='lines', 
                         hoverinfo='none', 
                         name='Regression Line', 
                         line=dict(dash='dot', color='rgba(255,0,0,0.2)')
                         ))

# Remove the legend
fig.update_layout(showlegend=False)


In [65]:
# Convert 'ds' to datetime format
df['ds'] = pd.to_datetime(df['ds'])

# Create a pivot table
pivot_table = df.pivot_table(values='y', index=df['ds'].dt.month, columns=df['ds'].dt.year, aggfunc='sum')

# Replace NaN values with 0
pivot_table.fillna(0, inplace=True)

# Format the pivot table
pivot_table.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
pivot_table.columns = pivot_table.columns.astype(str)
pivot_table = pivot_table.rename_axis("Report Count", axis="columns")
styled_table = pivot_table.round(2).style.format("{:,.0f}")

styled_table

Report Count,2019,2020,2021,2022
Jan,"1,000,001","48,061,004","96,189,298","149,727,574"
Feb,"2,984,797","49,280,673","96,189,299","150,817,985"
Mar,"2,984,799","49,346,529","96,228,303","150,817,986"
Apr,"2,984,801","49,348,136","96,228,304","151,271,376"
May,"2,984,875","50,529,476","96,228,306","151,271,378"
Jun,"2,984,877","50,529,568","96,228,309","151,271,381"
Jul,"4,244,474","50,787,433","96,228,312","153,120,398"
Aug,"4,244,476","50,787,493","96,228,372","153,748,728"
Sep,"4,244,477","51,278,430","96,228,374","154,280,058"
Oct,"47,374,196","92,943,849","149,167,453","172,212,537"
